In [2]:
# Packages einlesen
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# Daten einlesen
data_hedonic = pd.read_csv("/Users/paulahofmann/Documents/Coding/Online-Review/DataBase/WithMeta/Meta_hedonic.csv")

## Text Preprocessing



In [4]:
import spacy

# Load the English language model
nlp = spacy.load("en_core_web_sm")

# Function to preprocess text using spaCy with lemmatization and lowercasing

def preprocess_text(text):
    # Process the text with spaCy
    doc = nlp(text)
    
    # Lemmatization and lowercasing
    processed_text = ' '.join([token.lemma_.lower() for token in doc])
    
    return processed_text




In [5]:
# Adding another column, that filters out stop words and punctuation/symbols
import string

# Define a set of stop words
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Function to preprocess text using spaCy
def preprocess_text_stops(text):
    # Process the text with spaCy
    doc = nlp(text)
    
    # Lemmatization, lowercasing, and removal of symbols, punctuation, and stop words
    processed_text = ' '.join([token.lemma_.lower() for token in doc if token.is_alpha and not token.is_stop and not token.is_punct])
    
    return processed_text

In [8]:
# Hedonic Products
# Using Function to preprocess text using spaCy

# Apply the preprocessing function to the "text" column
data_hedonic['text_cleaned'] = data_hedonic['text'].apply(preprocess_text)

# Apply the preprocessing function with stopword removal to the "text" column
data_hedonic['text_cleaned1'] = data_hedonic['text'].apply(preprocess_text_stops)



KeyError: "['title'] not in index"

## Using Sentiment Analysis for Review Text

In [11]:
# Download the pre-trained BERT model and tokenizer
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

/opt/anaconda3/envs/torch-gpu/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# Load the pre-trained BERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("LiYuan/amazon-review-sentiment-analysis")
sentiment_model = AutoModelForSequenceClassification.from_pretrained("LiYuan/amazon-review-sentiment-analysis")


In [13]:
# Build Function to analyze the sentiment of a text

def analyze_sentiment(text):
    # Tokenize the text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    # Perform sentiment analysis
    outputs = sentiment_model(**inputs)
    # Get the predicted label
    predicted_label = torch.argmax(outputs.logits)
    return predicted_label.item()  # Return the predicted label as an integer


In [14]:
# Apply the function to the utilitarian product
data_hedonic['sentiment'] = data_hedonic['text'].apply(analyze_sentiment)


In [15]:
data_hedonic.to_csv('data_hedonic.csv', index=False)


## Topic Modelling

In [25]:
from bertopic import BERTopic

In [27]:
# Importing BErt Topic Model
modelNegative = BERTopic(language="english", nr_topics="auto")

data_utilitarian_filter['text_cleaned1'] = data_utilitarian_filter['text_cleaned1'].fillna('').astype(str)


# Topic Modelling for Hedonic Products
topics, probabilities = modelNegative.fit_transform(data_utilitarian_filter['text_cleaned1'])


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [33]:
# Getting only negative Comments:
dataNegative = data_utilitarian_filter[data_utilitarian_filter.text_sentiment<3]

topics, probabilities = modelNegative.fit_transform(dataNegative['text_cleaned1'])

reviewsNegativeText = dataNegative.text.to_list()

modelNegative.get_topic_freq().head(11)


,Topic,Count
0,0,451
1,-1,390
4,1,78
2,2,40
5,3,23
6,4,21
3,5,12


In [30]:
modelNegative.get_topic(1)


[('work', 0.17938184128795043),
 ('expensive', 0.15326731453794792),
 ('month', 0.1426010963332413),
 ('wish', 0.10679951737809838),
 ('cheap', 0.1067586253042321),
 ('need', 0.10511226304922043),
 ('dislike', 0.0986708493300253),
 ('overprice', 0.0733591839194295),
 ('job', 0.06030037295769011),
 ('money', 0.05823197743867206)]

In [31]:
modelNegative.get_topic(2)


[('expensive', 1.1142568839520837),
 ('little', 0.22867789388745127),
 ('cheap', 0.14895571040471908),
 ('need', 0.14665861208053285),
 ('trust', 0.14123423891708398),
 ('suppose', 0.13564000611888638),
 ('version', 0.11741684243451618),
 ('janky', 0.08567323357904423),
 ('ink', 0.08567323357904423),
 ('dislke', 0.08567323357904423)]

In [32]:
modelNegative.visualize_barchart()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.03803007489580293, 0.03834874232034199,
                    0.039666381542523235, 0.07705096146740537, 0.08162011200078698],
              'xaxis': 'x',
              'y': [buy  , work  , taste  , water  , filter  ],
              'yaxis': 'y'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.1067586253042321, 0.10679951737809838, 0.1426010963332413,
                    0.15326731453794792, 0.17938184128795043],
              'xaxis': 'x2',
              'y': [cheap  , wish  , month  , expensive  , work  ],
              'yaxis': 'y2'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.14123423891708398, 0.14665861208053285, 0.14895571040471908,
                    0.22867789388745127, 1.1142568839520837],
              'xaxis': 'x3',
              'y': [trust  , need  , cheap  , little  , expensive  ],
              'yaxis': 'y3'},
             {'marker': {'color': '#E69F00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.10952097577306806, 0.11084953263993214, 0.12055438701338496,
                    0.1882513891583287, 0.20747020872370955],
              'xaxis': 'x4',
              'y': [expensive  , fit  , fridge  , whirlpool  , refrigerator  ],
              'yaxis': 'y4'},
             {'marker': {'color': '#56B4E9'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.19224237778712366, 0.22063341454580662, 0.2723289827453242,
                    0.51663585077103, 0.6692451363956889],
              'xaxis': 'x5',
              'y': [unknown  , work  , design  , problem  , complaint  ],
              'yaxis': 'y5'},
             {'marker': {'color': '#009E73'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.13595164044875155, 0.14504913680152615, 0.18428999901322185,
                    0.20313315601512832, 0.22153746612218828],
              'xaxis': 'x6',
              'y': [cheap  , en  , el  , que  , de  ],
              'yaxis': 'y6'}],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 0',
                                'x': 0.0875,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 1',
                                'x': 0.36250000000000004,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 2',
                                'x': 0.6375000000000001,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'},
                               {'font': {'size': 16},
                                'showarrow': False,
                                'text': 'Topic 3',
                                'x': 0.9125,
                                'xanchor': 'center',
                                'xref': 'paper',
                             